In [ ]:
def dp_steam(inlet, outlet, pipe, pf):
    
    # Initialize; calculate inlet thermody properties
    inlet.steam_p()
    inlet.calc_area()    
    inlet.calc_velocity()
    outlet.calc_area() 

    '''Calculate Relative Roughness of the pipeline'''
    pipe.rel_roughness(inlet.d)
    
    '''Calculate Reynolds Number at the inlet node'''
    inlet.nRE()
    
    '''Calculate friction factor of the pipe'''
    if inlet.reynolds > 0: 
        pipe.ff_fanning(inlet.reynolds)
    else:
        pass    
    
    '''Calculate skin friction along the pipe'''
    if inlet.reynolds > 0: 
        pipe.skin_friction(inlet)
    else:
        pipe.f_skin = 0
    
    '''Calculate expansion or contraction friction loss'''
    outlet.area =  math.pi * outlet.d**2 / 4
    if inlet.area == outlet.area:
        pipe.f_exp = 0
        pipe.f_con = 0
    elif inlet.area < outlet.area:
        pipe.f_con = 0
        pipe.f_expansion(inlet, outlet)
    else:
        pipe.f_exp = 0
        pipe.f_contraction(inlet, outlet)

    '''Calculate Friction Loss from fittings'''
    pipe.f_fittings(kf_list, inlet.v)
    
    '''Calcutate total friction loss'''
    pipe.f_total()
    
    '''Calculate Heat Loss'''
    pipe.q_loss(inlet, insul)

    '''Calculate gdz and udu'''
    outlet.v = inlet.mf / (inlet.rho * outlet.area) #temp outlet v; using inlet params
    pipe.f_gdz(inlet.z, outlet.z)
    pipe.f_udu(inlet.v, outlet.v)

    '''Total Loss'''
    pipe.total_loss()

    '''Iterate until convergence to calculate outlet pressure'''
    #Mass Flow, Enthalpy and T_sat are also calculated at the end
    iter_dp(inlet, outlet, pipe, pf)
        
    '''Calculate Exergy at Inlet and Outlet'''
    outlet.calc_exergy()
    inlet.calc_exergy() 

    '''Set SiO2 and SSI at Zero'''
    outlet.sio2 = 0
    outlet.ssi = 0

    '''Calculate CO2 and H2S of steamline outlet node'''
    outlet.co2_h2s(inlet, pf)
    
    '''Calculate NCG for steam lines'''
    if pipe.is_stl():
        outlet.calc_ncg()
    else:
        outlet.ncg = 0
    
    '''Print Node and Pipe Parameters'''
    print(pipe.ID, pipe.is_stl(), pf)
    print_f(inlet, outlet, pipe)